## **CA2-AI**
# **Amir-Hossein Shahgholi(810199441)**

### Part1-Genetic

In first part we want to implementing a genetic algorithm for portfolio management:

At first we should create our Portfolio class to representation our chromosomes.

In [10]:
class Portfolio:
    data = list
    fitness_result = int
    
    def __init__(self, data):
        self.data = data

In our Chromosome class we have a list of data that represent amount of each shares in our Protfolio.

Now we should create our population first. We fill data with some random numbers that sum of them will be 1.
To do this we first generate some random numbers and then normalize the numbers.</br>
Below function will do that for us.

In [11]:
import random

ZERO_PERCENT = 0.6
def create_sample_data(number_of_data):
    numbers = []
    for _ in range(number_of_data):
        if (random.random()< ZERO_PERCENT):
            numbers.append(0)
        else:
            random_no = random.random()
            numbers.append(random_no)
    total = sum(numbers)
    numbers = [x/total for x in numbers]
    return numbers

So we create sample portfolio with randoms data that represent our population.
ٔNumber of Choromosomes in our population set as 1000.

In [12]:
NUMBER_OF_CHOROMOSOME_IN_POPULATION = 1000
NUMBER_OF_SHARES = 400

population = []

for _ in range(NUMBER_OF_CHOROMOSOME_IN_POPULATION):
    sample_data = create_sample_data(NUMBER_OF_SHARES)
    temp_portfolio = Portfolio(data=sample_data)
    population.append(temp_portfolio)

In the next step we should define a fitness function to calculate a fitness value for each portfolio.
For fitness function first we find total_returns of our portfolio and then we should find risk of our portfolio using the portfolio's standard deviation and then we calculate a fitness value with this formula: **fitness = (portfolio_return - risk_penalty * portfolio_risk) / return_penalty**

In [13]:
import numpy as np

risk_penalty = 0.6
return_penalty = 10


def calculate_fitness(returns: list, risks: list, portfolio):
    fitness = 0
    returns = np.array(returns, dtype=np.float64)
    risks = np.array(risks, dtype=np.float64)
    shares_percent = np.array(portfolio.data, dtype=np.float64)
    num_zeros = np.count_nonzero(shares_percent == 0.0)
    if (NUMBER_OF_SHARES - num_zeros) <30:
        fitness -= 0.1
    portfolio_return = np.dot(returns, shares_percent)
    portfolio_risk = np.sqrt(np.dot(shares_percent**2, risks))
    fitness += (portfolio_return - risk_penalty * portfolio_risk) / return_penalty
    return fitness

To find crossover of two chromosomes we use uniform crossover and then after finding child's data we normalize the data to have some of one in child's data. 

In [14]:
def crossover(portfolio1, portfolio2) -> Portfolio:
    pdata1 = portfolio1.data 
    pdata2 = portfolio2.data
    child_data = []
    for i in range(NUMBER_OF_SHARES):
        if random.randint(0, 1) == 0:
            child_data.append(pdata1[i])
        else:
            child_data.append(pdata2[i])
    total = sum(child_data)
    child_data = [x/total for x in child_data]
    return Portfolio(child_data)

And we define a function for mutation that get portfolio data and replace the portfolio's data with some new data.

In [15]:
def mutation(portfolio) -> Portfolio:
    portfolio.data = create_sample_data(len( portfolio.data))
    return portfolio

Read the sample data using `csv` library and save returns and risk of shares in two lists.

In [16]:
import csv

returns = []
risks = []

with open('sample.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        returns.append(row['return'])
        risks.append(row['risk'])

We define a function to print data of portfolio.

In [17]:
def print_portfolio_data(portfolio, returns, risks):
    returns = np.array(returns, dtype=np.float64)
    risks = np.array(risks, dtype=np.float64)
    shares_percent = np.array(portfolio.data, dtype=np.float64)
    portfolio_return = np.dot(returns, shares_percent)
    portfolio_risk = np.sqrt(np.dot(shares_percent**2, risks))
    print(f"Fitness: {portfolio.fitness_result}")
    print(f"Return: {portfolio_return}")
    print(f"Risk: {portfolio_risk}")
    print(f"Data: {portfolio.data}")

In this main step we create new generations. First we calculate the fitness for all choromosomes in our populationa and then we sort them, in next part we going into a loop that generate new generations in each loop.
In each loop we run the crossover function with probability of PC and mutation with probability of PM. After crossover we delete the choromosome with lowest fitness and then apend new choromosome to the population and when we got expected fitness we break the loop.

In [18]:
PC = 0.9
PM = 0.05
for cal in population:
    cal.fitness_result = calculate_fitness(returns, risks, cal)
population = sorted(population, key=lambda portfolio: portfolio.fitness_result)
while True:
    if random.random() <= PC:
        population_no = len(population)
        parents = [population[i] for i in random.choices(range(population_no), weights=range(population_no), k=2)]
        new_child = crossover(parents[0], parents[1])
        new_child.fitness_result = calculate_fitness(returns, risks, new_child)
        population.pop(0)
        for z, chrom in enumerate(population):
            if chrom.fitness_result > new_child.fitness_result:
                population.insert(z, new_child)
                break
        else:
            population.append(new_child)
    if random.random() <= PM:
        random_chrom_index = random.randint(0,len(population)-1)
        population[random_chrom_index] = mutation(population[random_chrom_index])
    if population[-1].fitness_result > 1:
        break
print_portfolio_data(population[-1], returns, risks)
    

Fitness: 1.0214343388752893
Return: 10.428378395544618
Risk: 0.3567250113195414
Data: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2020110365422299, 0.0036807473854763278, 0.0, 0.0008530190963203126, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000427016102655134, 0.0, 0.0, 0.0, 0.003099232546167516, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0009202556978802475, 0.00011942068617053805, 0.0, 0.0014749909878485046, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05126551118227019, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.013554493559048277, 0.0, 0.0, 0.0, 0.08649740724734112, 0.0, 0.0, 0.0, 0.0, 0.003411375156293498, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005163946652038357, 0.0, 0.003380277844291648, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

### Answer a questions:

**1-** The small population can result in the algorithm getting stuck in a local optimum answer and not finding the global optimum answer. In large population finding the answer take long time and can result in longer computation times, which can be a problem for optimization problems with large scales.

**2-** It increases the computation times and the memory usage very much and unnecessary data(like choromosomes with low fitness) take large amount of memmory.

**3-** With crossover combining information from two or more parents to create better offsprings and mutation introducing random changes to the offspring to create some new genes. We need both of them for example crossover without mutation can getting stuck in a local answer and new childs don't getting better in new generations.

**4-** Chosing a good Fitness function, the initial population size and their data should be chosen carefully to create variant init data and parameters of the probability of crossover and mutation are helpful to make iit faster.

**5-** The algorithm may find local optimum answer and not finding the global optimum and that answer may not be good for our fitness. In this cases we can use mutation with more probability and also we can stop the loop if in new generation our fitness not growing.

**6-** As said in above question we can break the loop if we don't have grow in our new generation's fitness.

## Part2-Game

In this part we should implement `minimax` algorithm for Othello game.

Sample game code is given to us and it's in main function so we first import main into our project.

In [1]:
import main

Now we add some new function to Othello class that provided.
First we add an `evaluate` function to evaluates the current state of the board for the given player. We consider several criteria for example:</br>
    - Pieces controlled by the player.</br>
    - Bonus for pieces in the corners.</br>
    - Penalty for pieces on the edge but not in the corners.</br>
    - And also consider penalty with same weight for pieces of opponent.</br></br>
And in next step we implement a `minimax` function to find score of each node in the tree using evaluate function.</br></br>

In last step we override the `get_human_move` methode to select a move using minimax.

In [2]:
from copy import deepcopy

class OthelloMinimax(main.Othello):
    def evaluate(self, player):
        score = 0
        score += sum([row.count(player) for row in self.board])
        return score
    
    def minimax(self, board, depth, player):
        self.number_of_seen_nodes += 1 # To count number of seen nodes
        if depth == 0 or self.terminal_test(): # If it's a leaf for our current tree then return score
            return self.evaluate(player)
        if player == 1: # Max function
            max_score = float('-inf')
            for move in self.get_valid_moves(player):
                temp_board = deepcopy(board)
                self.make_move(player, move)
                score = self.minimax(temp_board, depth-1, -player)
                self.board = deepcopy(temp_board)
                max_score = max(max_score, score)
            return max_score
        else: # Min function
            min_score = float('inf')
            for move in self.get_valid_moves(player):
                temp_board = deepcopy(board)
                self.make_move(player, move)
                score = self.minimax(temp_board, depth-1, -player)
                self.board = deepcopy(temp_board)
                min_score = min(min_score, score)
            return min_score
        
    def get_human_move(self):
        moves = self.get_valid_moves(1)
        if len(moves) == 0:
            return None
        best_move = None
        best_score = float('-inf')
        for move in moves:
            temp_board = deepcopy(self.board)
            self.make_move(1, move)
            score = self.minimax(temp_board, self.minimax_depth-1, -1)
            self.board = deepcopy(temp_board)
            if score > best_score:
                best_move = move
                best_score = score
        return best_move

Now our Othello getting complete with minimax algorithm and specific depth and we run our tests.

In [3]:
def run_test(depth, number_of_tests):
    total_time = 0
    total_seen_nodes = 0
    win_count = 0
    for _ in range(number_of_tests):
        o = OthelloMinimax(ui=False, minimax_depth=depth)
        result = o.play()
        if result[0] == 1:
            win_count += 1
        total_time += result[1]
        total_seen_nodes += result[2]
    print(f"Wins percent:{win_count/number_of_tests*100}")
    print(f"Avg executation time:{total_time/number_of_tests}")
    print(f"Avg visited nodes:{total_seen_nodes/number_of_tests}")


First we test with depth one.

In [7]:
run_test(depth=1, number_of_tests=200)

Wins percent:38.0
Avg executation time:0.008245404958724976
Avg visited nodes:85.205


We increase depth to 3.

In [90]:
run_test(depth=3, number_of_tests=100)

Wins percent:57.99999999999999
Avg executation time:0.3449984097480774
Avg visited nodes:3606.76


And depth 5.
we decrease number_of_tests also because it takes long time to run.

In [45]:
run_test(depth=5, number_of_tests=100)

Wins percent: 64.0
Avg execution time: 62.503296823501586
Avg visited nodes: 169697.94


Now we implement minimax with alpha beta and prune.

In [4]:
from copy import deepcopy

class OthelloMinimaxA(main.Othello):
    def evaluate(self, player):
        score = 0
        score += sum([row.count(player) for row in self.board])
        return score
    
    def alpha_beta_minimax(self, depth, player, alpha, beta):
        self.number_of_seen_nodes += 1
        if depth == 0 or self.terminal_test():
            return sum([row.count(1) for row in self.board])
        if player == -1:
            best_score = float('inf')
            for move in self.get_valid_moves(player):
                board_copy = [row[:] for row in self.board]
                self.make_move(player, move)
                score = self.alpha_beta_minimax(depth - 1, 1, alpha, beta)
                self.board = board_copy
                best_score = min(best_score, score)
                beta = min(beta, score)
                if alpha >= beta:
                    break
            return best_score
        else:
            best_score = float('-inf')
            for move in self.get_valid_moves(player):
                board_copy = [row[:] for row in self.board]
                self.make_move(player, move)
                score = self.alpha_beta_minimax(depth - 1, -1, alpha, beta)
                self.board = board_copy
                best_score = max(best_score, score)
                alpha = max(alpha, score)
                if alpha >= beta:
                    break
            return best_score
        
    def get_human_move(self):
        moves = self.get_valid_moves(1)
        if len(moves) == 0:
            return None
        best_move = None
        best_score = float('-inf')
        for move in moves:
            temp_board = deepcopy(self.board)
            self.make_move(1, move)
            score = self.alpha_beta_minimax(self.minimax_depth-1, -1, float('-inf'), float('inf'))
            self.board = deepcopy(temp_board)
            if score > best_score:
                best_move = move
                best_score = score
        return best_move

In [12]:
def run_testA(depth, number_of_tests):
    total_time = 0
    total_seen_nodes = 0
    win_count = 0
    loop_counter = 0
    while loop_counter<number_of_tests:
        o = OthelloMinimaxA(ui=False, minimax_depth=depth)
        result = o.play()
        if result == (0,0,0):
            continue
        if result[0] == 1:
            win_count += 1
        total_time += result[1]
        total_seen_nodes += result[2]
        loop_counter+=1
    print(f"Wins percent:{win_count/number_of_tests*100}")
    print(f"Avg executation time:{total_time/number_of_tests}")
    print(f"Avg visited nodes:{total_seen_nodes/number_of_tests}")

And we test new Othello with some depth and see the result.

In [6]:
run_testA(depth=1, number_of_tests=200)

Wins percent:63.0
Avg executation time:0.005793116092681885
Avg visited nodes:78.9


In [7]:
run_testA(depth=3, number_of_tests=100)

Wins percent:73.0
Avg executation time:0.06678219079971313
Avg visited nodes:1801.72


In [8]:
run_testA(depth=5, number_of_tests=100)

Wins percent:92.0
Avg executation time:1.0222524881362915
Avg visited nodes:25717.03


In [13]:
run_testA(depth=7, number_of_tests=100)

Wins percent:97.0
Avg executation time:7.864392778873444
Avg visited nodes:185928.8


As we can see in above tests it's really faster than one without prune and even more wins percent.


### Answer Questions

**1-** First it should be quick to compute and not require too much memory and also should provide accurate estimates of the cost of reaching the goal state.

**2-** They are clearly related to each other, If we increase the depth as we expected number of visited nodes and execution time increase but it's good because our wins-percent in also increase and it's because we check more steps and make a more accurate decision.

**3-** Yes, We can use a heuristic that estimates the expected value of each child node, and then arrange the children in descending order of their expected value and it's help us to have more prune in travel the tree.

**4-** It is the average number of child nodes for each node in a search tree that we have. At the start of the game, the branching factor is very high, as there are many possible moves that a player can make. As the game progresses and more spaces become occupied, the number of available moves decreases, and the branching factor decreases too.

**5-** This is because it eliminates the need to explore the branches that do not lead to the solution. As a result, the algorithm can focus on the more promising branches and explore them in more detail, which leads to a faster and more accurate solution.

**6-** It assumes that both players are playing optimally and will choose the best move at each turn. In random play, the players do not always choose the best move, so the minimax algorithm will not accurately predict the outcome of the game. As i searched one algorithm that can be used for random play is Monte Carlo Tree Search (MCTS). MCTS is a heuristic search algorithm that is based on repeatedly simulating the play of the game from the current state to the end of the game. At each simulation, the algorithm selects a move to make based on the current state of the game, and then plays the game out randomly from that move to the end.